# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Разберемся, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — ответить на четыре вопроса:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о платежеспособности клиентов получим из файла `data.csv`. Нам понадобится обзор данных перед проверкой гипотез, так как об их качестве сейчас нет информации. Мы проверим данные на наличие ошибок и не только, а на этапе предобработки поищем возможность все исправить. Затем сделаем вывод по исследованию. 
 
Таким образом, исследование пройдёт в четыре шага:
 1. Обзор данных.
 2. Предобработка данных.
 3. Ответы на вопросы.
 4. Общий вывод.

## Шаг 1. Обзор данных

Импортируем библиотеку `pandas` и сохраним ее в сокращенном варианте `pd`. Затем прочитаем файл `data.csv` из папки `/datasets`, сохранив его в переменной `customers`:

In [1]:
import pandas as pd
customers = pd.read_csv('/datasets/data.csv')

Выведем на экран первые пятнадцать строк таблицы:

In [2]:
display(customers.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


И дополнительно познакомимся с данными таблицы при помощи вызова `info()`:

In [3]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в таблице 12 столбцов. Тип данных при этом различается. Есть `float`, `int` и `object`. Согласно документации к данным:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

В названиях колонок не видно нарушений стиля, поэтому что-либо корректировать в них не будем. Везде соблюден «змеиный_регистр». Однако количество значений в столбцах различается. Значит, в данных есть пропущенные значения. В столбце `education` категории нужно свести к нижнему регистру, чтобы результаты анализа не искажались. Еще в столбце `days_employed` какие-то аномальные значения с отрицательным количеством дней трудового стажа. Необходимо будет привести эти данные к должному виду. 

**Вывод**

В каждой строке таблицы — данные о потенциальном заёмщике. Большинство колонок описывает именно самого заёмщика, лишь в последнем речь о цели получения кредита. 

Предварительно можем утверждать, что данных достаточно для ответов на вопросы, однако есть пропуски в данных и нужно проверить соответстсвие типа данных с данными в столбце. Устраним эти недочеты, попробуем применить другие фишки для предобработки данных и потом будем двигаться дальше. 

## Шаг 2. Предобработка данных

Исключим пропущенные значения, заменим тип данных в столбцах и обработаем дубликаты. Затем применим категоризацию данных. 

### 2.1 Обработка пропусков

Посчитаем, сколько в таблице пропущенных значений:

In [4]:
# подсчёт пропусков
customers.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Не все пропуски повлияют на исследование. В `days_employed` общий трудовой стаж в днях, который не важен для нашей работы, поэтому достаточно заменить эти пропуски на явные обозначения. Однако пропуски в `total_income` повлияют, так как для ответа на один из вопросов нужны данные по доходу заёмщика. Придется заполнить эти пропуски. Если в пропущенных значениях `days_employed` позволительно указать явные обозначения, то в случае с `total_income` придется указывать медиану дохода с группировкой по типу занятости. 

Сначала сразу приведем столбец `days_employed` в нормальный вид, так как сейчас там наблюдаются отрицательные и аномальные значения. Учитывая, что есть и нормальные значения, скорее всего речь идет о получении данных из разных источников, где значения были приведены в разных величинах. Приведем данные к положительным значениям и поделим их на 24, так как предположительно они указаны в часах. 

In [5]:
#метод apply возьмет значения по модулю
customers['days_employed'] = customers['days_employed'].apply(abs)
#определим предположительную границу того, что переводим в часы, а что нет, и затем нужное делим на 24
customers.loc[customers['days_employed'] >
              30000, 'days_employed'] = customers.loc[customers['days_employed'] >
                                                      30000, 'days_employed'] / 24

Выясним, в одинаковых ли строках пропущены значения по обоим столбцам:

In [6]:
customers[(customers['total_income'].isnull() == True) & (customers['days_employed'].isnull() == True)].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Получается, что в обоих столбцах пропущены значения только в одинаковых строках. Для начала найдем среднее значение стажа и заполним им пропуски в столбце `days_employed`:


In [7]:
days_employed_mean = customers['days_employed'].mean()
customers['days_employed'] = customers['days_employed'].fillna(days_employed_mean)

В случае с доходами простой медианы будет недостаточно, так как доходы скорее всего зависят от уровня образования клиентов. Возьмем зависимость доходов от уровня образования по столбцу `education_id` и заполним пропуски с группировкой:

In [8]:
for education_id in customers['education_id'].unique():
    median = customers.loc[customers['education_id'] == education_id, 'total_income'].median()
    customers.loc[(customers['total_income'].isna()) &
                  (customers['education_id'] == education_id), 'total_income'] = median

Убедимся, что в данных отсутствуют пропуски:

In [9]:
customers.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

В стаже отрицательные значения привели к положительным и поделили на 24, приведя их к нормальному виду. Пропуски заменили на среднее и медианное значение, где это требовалось. Возможно, что пропущенные значения образовались в результате какого-то сбоя при выгрузке данных, либо из-за человеческого фактора. 

### 2.2 Замена типа данных

В столбцах `days_employed` и `total_income` тип данных `float`, заменим его на `int`, чтобы получить целые числа:

In [10]:
customers['days_employed'] = customers['days_employed'].astype('int')
customers['total_income'] = customers['total_income'].astype('int')

И убедимся в том, что тип данных успешно изменен:

In [11]:
customers.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

**Вывод**

Метод `to_numeric()` в этом случае не сработает, так как значения в столбцах уже в числовом формате, поэтому для изменения типа данных используем метод `astype()` с аргументом `int`.

### 2.3 Обработка дубликатов

Для начала избавимся от неявных дубликатов в колонке `education`. Значение образования указано разными регистрами. Это может повлиять на результаты исследования в том случае, если начать другое исследование, с вопросом о том, влияет ли уровень образования на возврат кредита в срок. Сейчас мы просто приведем значения в нормальный вид. Поиск неявных дубликатов всегда начинают с метода `unique()`, на нем и остановимся. 

In [12]:
# просмотр уникальных названий уровня образования
sorted(customers['education'].unique())

['ВЫСШЕЕ',
 'Высшее',
 'НАЧАЛЬНОЕ',
 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
 'Начальное',
 'Неоконченное высшее',
 'СРЕДНЕЕ',
 'Среднее',
 'УЧЕНАЯ СТЕПЕНЬ',
 'Ученая степень',
 'высшее',
 'начальное',
 'неоконченное высшее',
 'среднее',
 'ученая степень']

Все неявные дубликаты можно исправить одним методом `lower()`, так как лишних пробелов или ошибок в самих названиях нет, соответственно достаточно установить единый регистр для всех значений:

In [13]:
customers['education'] = customers['education'].str.lower()

Еще раз проверим уникальные названия, чтобы убедиться в отсутствии неявных дубликатов столбца `education`:

In [14]:
sorted(customers['education'].unique())

['высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']

Теперь посчитаем количество явных дубликатов. В этом нам поможет специальный метод `duplicated()`. Он возвращает Series со значением True для строк-дубликатов, а чтобы посчитать такие строки, результат метода передают функции `sum()`. 

In [15]:
# подсчёт явных дубликатов
customers.duplicated().sum()

71

Вызовем специальный метод `pandas`, чтобы удалить явные дубликаты. Речь о `drop_duplicates()`, только он сможет нам сейчас помочь:

In [16]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
customers = customers.drop_duplicates().reset_index(drop = True)

Еще раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [17]:
# проверка на отсутствие дубликатов
customers.duplicated().sum()

0

**Вывод**

Похоже на то, что в анкете банка нет фиксированного формата ввода информации по образованию, а также нет какого-то фильтра этой информации перед попаданием в базу данных. Клиенты сами пишут уровень образования разным регистром, поэтому появляются такие неявные дубликаты. Было бы полезно передать ответственным информацию о том, чтобы при заполнении анкеты регистр автоматически менялся на нужный, тогда можно будет избежать такой ситуации.

Учитывая, что явных дубликатов было немного, возможно, что речь о человеческом факторе, либо небольшом задвоении данных при выгрузке. 

### 2.4 Лемматизация

В таблице есть столбец `purpose` с данными о цели получения кредита. Можно заметить, что есть похожие друг на друга цели, просто выраженные разными словами. Найдем количество уникальных лемм и добавим еще один столбец с категорией о цели получения кредита. Сначала импортируем библиотеку `pymystem3`:

In [18]:
from pymystem3 import Mystem
m = Mystem()

Выведем список уникальных целей и их количество:

In [19]:
print(customers['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Лемматизируем полученный выше список, чтобы получить список уникальных лемм и отобразим их количество:

In [20]:
list_unique = customers['purpose'].unique()
list_unique = ','.join(list_unique)
lemmas = m.lemmatize(list_unique)
from collections import Counter
print(Counter(lemmas)) 

Counter({' ': 59, ',': 37, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Выделим вручную основные ключевые слова и заключим их в список:

In [21]:
purpose_categories = ["образование", "свадьба", "недвижимость", "автомобиль", "жилье"]

Создадим функцию для проведения лемматизации и одновременно заменим полученный список лемм в каждой строке на ключевое слово из списка категорий:

In [22]:
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in purpose_categories:
        if word in lemma:
            lemma = word
    return lemma

customers['purpose_categories'] = customers['purpose'].apply(lemmatize)

Теперь убедимся, что категрий стало меньше и они более точечны, а также в отсутствии лишних категорий:

In [23]:
print(customers['purpose_categories'].value_counts())
customers.head(5)

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_categories, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categories
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

В целях получения кредита снова были данные, заполненные клиентами самостоятельно в анкете, поэтому они различались по виду, но не по смыслу. Внедрение выпадающего списка в анкете помогло бы избежать этой работы вообще. Но сейчас это не так, поэтому мы лемматизировали данные, чтобы получить 5 точечных категорий и добавили отдельный столбец в таблицу. Для исследования это не нужно было, но помогло бы для других задач или если бы за работу с этими данными взялся другой аналитик.  Дополнительно убедились в том, что все категории добавились правильно. 

### 2.5 Категоризация данных

Создадим функцию для категоризации уровня дохода, показатели категорий доходов установим сами:

In [24]:
def replace_income(total_income):
    if total_income >= 0 and total_income <= 50000:
        return 'низкий'
    if total_income > 50000 and total_income <= 100000:
        return 'средний'
    if total_income > 100000 and total_income <= 150000:
        return 'выше среднего'
    if total_income > 150000:
        return 'высокий'

customers['total_income_id'] = customers['total_income'].apply(replace_income)

И убедимся, что функция работает:

In [25]:
customers.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categories,total_income_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,выше среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий


In [26]:
customers['total_income_id'].value_counts()

высокий          9787
выше среднего    7204
средний          4091
низкий            372
Name: total_income_id, dtype: int64

Теперь создадим функцию для категоризации по возрасту. Здесь пороги показателей для определенной категории также установим самостоятельно:

In [27]:
def replace_age(dob_years):
    if dob_years <= 30:
        return 'до 30 лет'
    if dob_years >= 31 and dob_years <= 54:
        return 'от 31 до 54 лет'
    if dob_years >= 55:
        return 'старше 55 лет'
customers['age_id'] = customers['dob_years'].apply(replace_age)

И также убедимся в том, что она работает и показывает правильные категории:

In [28]:
customers.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categories,total_income_id,age_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,высокий,от 31 до 54 лет
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,выше среднего,от 31 до 54 лет
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,выше среднего,от 31 до 54 лет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий,от 31 до 54 лет
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий,от 31 до 54 лет


**Вывод**

Пороги для категорий в всех случаях установили самостоятельно, так как они могут различаться из официальных источников как по регионам, так и по другим факторам. Сейчас главное отобразить общую картину, затем при необходимости их можно будет скорректировать в зависимости от того, где именно будут использовать данные. 



Отдельно обнаружим и выделим словари, которые есть в данных. При необходимости будем работать с ними:

In [29]:
dictionaries = customers[['education', 'education_id', 'family_status', 'family_status_id']]
display(dictionaries.head(10))

,education,education_id,family_status,family_status_id
0,высшее,0,женат / замужем,0
1,среднее,1,женат / замужем,0
2,среднее,1,женат / замужем,0
3,среднее,1,женат / замужем,0
4,среднее,1,гражданский брак,1
5,высшее,0,гражданский брак,1
6,высшее,0,женат / замужем,0
7,среднее,1,женат / замужем,0
8,высшее,0,гражданский брак,1
9,среднее,1,женат / замужем,0


In [30]:
customers[['education', 'education_id']].drop_duplicates()

,education,education_id
0,высшее,0
1,среднее,1
13,неоконченное высшее,2
31,начальное,3
2962,ученая степень,4


In [31]:
customers[['family_status', 'family_status_id']].drop_duplicates()

,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,Не женат / не замужем,4


## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Составим функцию, которая поможет нам определить соотношение категории наличия детей и уровнем дохода к возврату кредита в срок:

In [32]:
def relation(category):
    return customers.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

relation('children')

,debt
children,
5,0.000000
-1,0.021277
0,0.075438
3,0.081818
1,0.092346
2,0.094542
4,0.097561
20,0.105263


**Вывод**

Зависимость есть. Как видно из результата, заемщики без детей имеют меньше всего проблем с возвратом кредита в срок, а вот наоборот состоит ситуация у заемщиков, у которых есть 1-2 ребенка. Можно предположить, что при наличии финансовых проблем клиент скорее предпочтет в первую очередь обеспечить своего ребенка, чем решить вопрос с банком. При этом именно с 3-мя детьми заемщики имеют меньше проблем с возвратом кредита в срок, чем у тех, кто имеет 1-2 ребенка. 

В данных есть отрицательное значение, скорее оно появилось в результате какой-то технической проблемы при переносе данных и речь идет об отсутствии детей. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Применим метод `.agg` к столбцу `family_status` для подсчета числа должников, общего числа клиентов в группах и нахождения доли должников в каждой из групп:

In [33]:
family_agg = customers.groupby('family_status').agg({'debt':['count', 'sum', 'mean']})
display(family_agg)

debt               
                       count  sum      mean
family_status                              
Не женат / не замужем   2810  274  0.097509
в разводе               1195   85  0.071130
вдовец / вдова           959   63  0.065693
гражданский брак        4151  388  0.093471
женат / замужем        12339  931  0.075452

**Вывод**

Зависимость есть. Удивительно, но именно вдовцы/вдовы и люди в разводе более ответственно подходят к возврату кредита в срок. Возможно, что это как-то связано с предыдущим опытом, так как именно не женатые/не замужние люди чаще всего себе позволяют просрочки. В том числе это касается пар, которые еще не узаконили свои отношения. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Теперь вызовем функцию к столбцу `total_income_id`, в котором мы создали категории уровня доходов:

In [34]:
relation('total_income_id')

,debt
total_income_id,
низкий,0.061828
высокий,0.077143
средний,0.080909
выше среднего,0.087729


**Вывод**

Клиенты с наименьшим доходом менее склонны к просрочкам, при этом по остальным категориям разброс небольшой и вряд ли можно сказать, что есть прямая зависимость уровня дохода и возвратом кредита в срок. Возможно, это связано с тем, что люди с наименьшим уровнем дохода ответственнее относятся к распределению своих денег, больше боятся за свой бюджет и как следствие внимательнее следят за возвратом кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

И в заключении создадим сводную таблицу, которая поможет ответить на последний вопрос:

In [35]:
purpose_pivot = customers.pivot_table(index = 'purpose_categories', values = 'debt', aggfunc = ['sum', 'count', 'mean'])
purpose_pivot.columns = ['debt', 'total', '%']
purpose_pivot = purpose_pivot.sort_values(by = ['total'], ascending = False)
display(purpose_pivot)

,debt,total,%
purpose_categories,,,
недвижимость,474,6351,0.074634
жилье,308,4460,0.069058
автомобиль,403,4306,0.093590
образование,370,4013,0.092200
свадьба,186,2324,0.080034


**Вывод**

Разница есть и заметная. Заемщики, которые берут кредит на недвижимость, более ответственны. Скорее всего это связано с тем, что в случае проблем с выплатой и затягиванияем решения вопроса, банк может через суд отобрать это имущество на реализацию. Странно, что с автомобилями такая гипотеза не работает, но и недвижимость, как правило, стоит в разы больше автомобиля. 

## Шаг 4. Общий вывод

**Вывод**

В общем и целом можно утверждать - наличие/отсутствие детей, семейное положение и уровень дохода определенно влияют на возврат кредита в срок. Мы ознакомились с имеющимися данными, провели тщательную предобработку, заполнив пропуски и категоризировав нужные группы, что позволило ответить на интересующие вопросы исследования. То есть, ответ на первые три вопроса - да, зависимость есть. Разные цели кредита также по разному влияют на возврат кредита в срок. 

Идеального клиента не существует, но если смотреть на результаты исследования, то наиболее ответственными являются клиенты, которые соответствуют этим параметрам:
- нет детей;
- вдовец/вдова;
- низкий доход;
- цель кредита - недвижимость. 

На основе этой информации скоринговую модель можно доработать, чтобы она более лояльно относилась к заемщикам с такими параметрами при проверке заявки, так как это в итоге снизит количество просрочек, как следствие снизит убытки банка из-за меньшего количества невозвратов и из-за меньших расходов на методы возврата долга.